# SHAPES OF OBJECTS
## This notebook outlines the concepts used in Shape detection of objects in an image

## Shape detection of Objects in Images


To find shapes of objects in an image:
- Find **contours** of objects
- Find **centers** of each object
- Recognize **shapes** of each object
    - Circle
    - Square
    - Rectangle
    - Triangle
    - Pentagon
- Label color of a shape

### Steps
- Import the necessary libraries
- Load the image
- Convert the image into Grayscale
- Blur the image
- Apply edge detection methods
    - Use Canny()
- Find contours
    - Use cv2.findContours()
- **Find centers of each contour**
- Draw the contours
- Display the drawn contours

#### Import the libraries

In [1]:
import cv2
import numpy as np

#### Load the image
![Shapes](https://raw.githubusercontent.com/subashgandyer/datasets/main/images/Shapes.png)

In [15]:
image = cv2.imread("Shapes.png")

#### Convert the image into Grayscale

In [16]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#### Blur the image

In [17]:
blurred = cv2.GaussianBlur(gray, (7, 7), 0)

#### Apply Canny Edge Detection

In [18]:
edged = cv2.Canny(blurred, 30, 150)

#### Display intermediate results

In [19]:
cv2.imshow("Shapes_Interim", np.hstack([gray, blurred, edged]))
cv2.waitKey(0)

13

#### Find Contours

**cv2.findContours()**
- image (recommeded to send its copy rather the original)
- type of contour
    - cv2.RETR_EXTERNAL
    - cv2.RETR_LIST
    - cv2.RETR_COMP
    - cv2.RETR_TREE
- approximation of contour
    - cv2.CHAIN_APPROX_SIMPLE
    - cv2.CHAIN_APPROX_NONE
- Returns a tuple
    - output image after applying contour detection
    - cnts list of contours detected
    - hierarchy of the contours

In [20]:
(cnts, _) = cv2.findContours(edged.copy(), 
                             cv2.RETR_EXTERNAL, 
                             cv2 .CHAIN_APPROX_SIMPLE
)

#### Compute the center of the contour
- cv2.moments()
    - m10 / m00 --> cX
    - m01 / m00 --> cY

In [21]:
M = cv2.moments(cnts[0])
cX = int(M["m10"] / M["m00"])
cY = int(M["m01"] / M["m00"])

#### Draw the Contours
cv2.drawContours()
- image
- contours list
- contour index
    - -1 --> draw all of the contours
    - i --> draw single contour
- color of the contour line
    - Use green color
- thickness of the line

cv2.circle()

cv2.putText()

In [9]:
shapes = image.copy()
cv2.drawContours(shapes, [cnts[0]], -1, (0, 255, 0), 2)
cv2.circle(shapes, (cX, cY), 7, (255, 255, 255), -1)
cv2.putText(shapes, 
            "center", 
            (cX - 20, cY - 20),
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.5, 
            (255, 255, 255), 
            2
)

array([[[22, 24, 24],
        [22, 24, 24],
        [22, 24, 24],
        ...,
        [22, 24, 24],
        [22, 24, 24],
        [22, 24, 24]],

       [[22, 24, 24],
        [22, 24, 24],
        [22, 24, 24],
        ...,
        [22, 24, 24],
        [22, 24, 24],
        [22, 24, 24]],

       [[22, 24, 24],
        [22, 24, 24],
        [22, 24, 24],
        ...,
        [22, 24, 24],
        [22, 24, 24],
        [22, 24, 24]],

       ...,

       [[50, 52, 52],
        [48, 50, 50],
        [50, 52, 52],
        ...,
        [41, 41, 41],
        [41, 41, 41],
        [40, 40, 40]],

       [[49, 51, 51],
        [48, 50, 50],
        [49, 51, 51],
        ...,
        [41, 41, 41],
        [41, 41, 41],
        [40, 40, 40]],

       [[50, 52, 52],
        [48, 50, 50],
        [49, 51, 51],
        ...,
        [41, 41, 41],
        [41, 41, 41],
        [40, 40, 40]]], dtype=uint8)

#### Display the Contours

In [10]:
cv2.imshow("Shapes", np.hstack([image, shapes]))
cv2.waitKey(0)

13

### Let's do it for all objects there

In [11]:
for c in cnts:
    M = cv2.moments(c)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    
    cv2.drawContours(shapes, [c], -1, (0, 255, 0), 2)
    cv2.circle(shapes, (cX, cY), 7, (255, 255, 255), -1)
    cv2.putText(shapes, 
                "center", 
                (cX - 20, cY - 20),
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.5, 
                (255, 255, 255), 
                2
    )
    
    cv2.imshow("Shapes", np.hstack([image, shapes]))
    cv2.waitKey(0)

## Shape detection

- **Contour Approximation**
    - Algorithm for reducing the number of points in a curve with a reduced set of points
    - Assumption: A curve can be approximated by a series of **short line segments**
- **cv2.approxPolyDP()**
    - contour
    - 1-15% of contour perimeter
- Shape analysis
    - If vertices are 3 in the contour
        - Traingle
    - If vertices are 5 in the contour
        - Pentagon
    - If vertices are 4 in the contour
        - If aspect ratio is between 0.95 to 1.05
            - Square
        - If aspect ration is skewed
            - Rectangle

In [22]:
class ShapeDetector:
    def __init__(self):
        pass
    
    def detect(self, c):
        # initialize the shape name and approximate the contour
        shape = "unidentified"
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        
        if len(approx) == 3:
            shape = "triangle"
        elif len(approx) == 4:
            # compute the bounding box of the contour & use the bounding box to compute the aspect ratio
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h)
            
            # a square will have an aspect ratio that is approximately
            # equal to one, otherwise, the shape is a rectangle
            shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"
            
        elif len(approx) == 5:
            shape = "pentagon"
        else:
            shape = "circle"
        
        return shape

In [23]:
sd = ShapeDetector()

### Change this for loop to accommodate the shape as the display text

In [24]:
for c in cnts:
    M = cv2.moments(c)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    
    shape = sd.detect(c)
    
    cv2.drawContours(shapes, [c], -1, (0, 255, 0), 2)
    cv2.putText(shapes, 
                shape, 
                (cX, cY),
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.5, 
                (255, 255, 255), 
                2
    )
    
    cv2.imshow("Shapes", np.hstack([image, shapes]))
    cv2.waitKey(0)

### Bonus Helper function to use old version codebase

### grab_contours( )

In [2]:
def grab_contours(cnts): 
    if len(cnts) == 2: 
        cnts = cnts[0]
    elif len(cnts) == 3:
        cnts = cnts[1]
    else:
        raise Exception(("Contours tuple must have length 2 or "
                        "3, otherwise OpenCV changed their cv2.findContours " 
                        "return signature yet again. "
                        "Refer to OpenCV’s documentation in that case."))
    return cnts